In [1]:
import os
import glob
import rasterio
from rasterio.windows import from_bounds, Window
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from PIL import Image
import pyproj
from pyproj import Transformer
import folium
from shapely.geometry import shape, box
from shapely.wkt import dumps
from shapely.wkt import loads
from shapely import wkt, affinity
from affine import Affine


def utm_to_latlon(easting, northing, crs):
    """Convert UTM coordinates to latitude and longitude using the CRS from the shapefile."""
    # Create CRS object
    utm_crs = pyproj.CRS(crs)
    latlon_crs = pyproj.CRS(proj='latlong', ellps='WGS84')
    
    # Create a transformer for converting coordinates
    transformer = Transformer.from_crs(utm_crs, latlon_crs, always_xy=True)
    lon, lat = transformer.transform(easting, northing)
    return lat, lon

def get_shapefile_bounds(shapefile_dir, output_csv_path, buffer_size=0):
    """Get filenames, geometry bounds, and geometries of shapefiles in a directory."""
    # Get all shapefiles in the directory
    shapefiles = glob.glob(os.path.join(shapefile_dir, '*.shp'))

    # Create an empty DataFrame with the required columns
    df = pd.DataFrame(columns=['Filename', 'MinX', 'MinY', 'MaxX', 'MaxY', 'Geometry'])

    for shapefile in shapefiles:
        # Read the shapefile
        gdf = gpd.read_file(shapefile)
        
        for _, row in gdf.iterrows():
            # Get the geometry and its bounds
            geometry = row['geometry'].buffer(buffer_size)
            minx, miny, maxx, maxy = geometry.bounds
            
            # Create a DataFrame for the current shapefile entry
            df_temp = pd.DataFrame([{
                'Filename': os.path.basename(shapefile),
                'MinX': minx,
                'MinY': miny,
                'MaxX': maxx,
                'MaxY': maxy,
                'Geometry': dumps(geometry)  # Convert geometry to WKT format
            }])
            
            # Concatenate the current shapefile DataFrame with the main DataFrame
            df = pd.concat([df, df_temp], ignore_index=True)

    # Write the DataFrame to a CSV file
    df.to_csv(output_csv_path, index=False)

    print(f"Results written to {output_csv_path}")

    return df

def get_tiff_bounds(tiff_dir, output_csv_path):
    """Get filenames and geometry bounds of TIFF files in a directory and output to a CSV file."""
    # Get all TIFF files in the directory
    tiff_files = glob.glob(os.path.join(tiff_dir, '*.tif'))

    # Create an empty DataFrame with the required columns
    df = pd.DataFrame(columns=['Filename', 'MinX', 'MinY', 'MaxX', 'MaxY'])

    for tiff_file in tiff_files:
        # Read the TIFF file
        with rasterio.open(tiff_file) as dataset:
            bounds = dataset.bounds  # bounds gives (left, bottom, right, top)
        
        # Create a DataFrame for the current TIFF file
        df_temp = pd.DataFrame([{
            'Filename': os.path.basename(tiff_file),
            'MinX': bounds.left,
            'MinY': bounds.bottom,
            'MaxX': bounds.right,
            'MaxY': bounds.top
        }])
        
        # Concatenate the current TIFF file DataFrame with the main DataFrame
        df = pd.concat([df, df_temp], ignore_index=True)

    # Write the DataFrame to a CSV file
    df.to_csv(output_csv_path, index=False)

    print(f"Results written to {output_csv_path}")
    return df

def check_shapefile_within_tiff(shapefile_bounds_df, tiff_bounds_df, output_csv_path):
    """Check if each shapefile is fully within the bounds of any TIFF file and write results to a CSV file."""
    results = []

    # Iterate through each shapefile's bounds
    for _, shapefile_row in shapefile_bounds_df.iterrows():
        shapefile_name = shapefile_row['Filename']
        shapefile_minx = shapefile_row['MinX']
        shapefile_miny = shapefile_row['MinY']
        shapefile_maxx = shapefile_row['MaxX']
        shapefile_maxy = shapefile_row['MaxY']
        
        is_within_any_tiff = False
        tiff_name_found = None
        
        # Check against each TIFF's bounds
        for _, tiff_row in tiff_bounds_df.iterrows():
            tiff_name = tiff_row['Filename']
            tiff_minx = tiff_row['MinX']
            tiff_miny = tiff_row['MinY']
            tiff_maxx = tiff_row['MaxX']
            tiff_maxy = tiff_row['MaxY']
            
            if (shapefile_minx >= tiff_minx and shapefile_miny >= tiff_miny and
                shapefile_maxx <= tiff_maxx and shapefile_maxy <= tiff_maxy):
                is_within_any_tiff = True
                tiff_name_found = tiff_name
                break  # No need to check further TIFFs once we find a match
        
        results.append({
            'Shapefile': shapefile_name,
            'Fully_Within_TIFF': is_within_any_tiff,
            'TIFF_File': tiff_name_found
        })
    
    # Create DataFrame from results
    results_df = pd.DataFrame(results)
    
    # Write the DataFrame to a CSV file
    results_df.to_csv(output_csv_path, index=False)

    print(f"Results written to {output_csv_path}")
    return results_df

def clip_and_save_tiff(shapefile_bounds_df, tiff_dir, search_df, output_dir):
    """Clip TIFF files to the bounds of shapefiles and save to the specified directory, returning a DataFrame of clipped TIFF bounds."""
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Check if the required columns are present
    required_shapefile_columns = {'Filename', 'Geometry'}
    required_search_columns = {'Shapefile', 'TIFF_File'}
    
    if not required_shapefile_columns.issubset(shapefile_bounds_df.columns):
        raise KeyError(f"Missing columns in shapefile_bounds_df: {required_shapefile_columns - set(shapefile_bounds_df.columns)}")
    
    if not required_search_columns.issubset(search_df.columns):
        raise KeyError(f"Missing columns in search_df: {required_search_columns - set(search_df.columns)}")
    
    # Create an empty list to store bounds of clipped TIFF files
    clipped_tiff_list = []
    
    for _, shapefile_row in shapefile_bounds_df.iterrows():
        shapefile_name = shapefile_row['Filename']
        shapefile_geometry = shapefile_row['Geometry']

        # Convert the string geometry to a shapely object
        geom = loads(shapefile_geometry)
        
        # Find the corresponding TIFF file
        tiff_file_row = search_df[search_df['Shapefile'] == shapefile_name]
        
        if tiff_file_row.empty:
            print(f"No matching TIFF file for shapefile {shapefile_name}")
            continue
        
        tiff_file_name = tiff_file_row['TIFF_File'].values[0]
        tiff_file_path = os.path.join(tiff_dir, tiff_file_name)
        output_file_path = os.path.join(output_dir, f"clipped_{shapefile_name}.tif")
        
        # Print debug information
        print(f"Processing shapefile {shapefile_name}...")
        print(f"Clipping TIFF file: {tiff_file_path}")
        print(f"Output clipped TIFF path: {output_file_path}")

        # Clip the TIFF file to the shapefile geometry and save
        clip_tiff_to_polygon(tiff_file_path, geom, output_file_path)
        
        # Record the bounds of the clipped TIFF file (bounding box for simplicity)
        clipped_tiff_list.append({
            'Filename': f"clipped_{shapefile_name}.tif",
            'Geometry': shapefile_geometry
        })
        
        print(f"Clipped TIFF saved to {output_file_path}")

    # Convert the list of clipped TIFF bounds to a DataFrame
    clipped_tiff_bounds_df = pd.DataFrame(clipped_tiff_list)

    return clipped_tiff_bounds_df

def clip_tiff_to_polygon(tiff_file, polygon_geom, output_file):
    """Clip a TIFF file to the specified polygon geometry and save the result."""
    
    with rasterio.open(tiff_file) as src:
        # Mask the image to the polygon
        out_image, out_transform = rasterio.mask.mask(src, [polygon_geom], crop=True)
        
        # Update metadata
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        
        # Write the clipped raster to the output file
        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(out_image)

def clip_and_save_large_tiff(input_dir, shapefile_path, tiff_df, output_path, buffer_distance=150):
    """Clip TIFF files based on shapefiles with a buffer distance and save the results.
    Returns a DataFrame with clipped TIFF filenames, bounding boxes, and geometries in WKT format."""
    
    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)
    
    # Load shapefiles
    shapefiles = glob.glob(os.path.join(shapefile_path, "*.shp"))
    
    # List to store results
    results = []
    
    # Iterate through each row in tiff_df
    for _, row in tiff_df.iterrows():
        tiff_file = row['TIFF_File']
        shapefile_name = row['Shapefile']
        
        # Construct the path for the shapefile
        shapefile_path_full = os.path.join(shapefile_path, shapefile_name)
        
        # Check if shapefile exists
        if not os.path.exists(shapefile_path_full):
            print(f"Shapefile {shapefile_path_full} does not exist. Skipping.")
            continue
        
        # Load the shapefile (assuming it contains only one polygon)
        shapefile = gpd.read_file(shapefile_path_full)
        if shapefile.empty:
            print(f"Shapefile {shapefile_name} is empty or not found.")
            continue
        shape_geom = shapefile.geometry.iloc[0]
        
        # Create rectangular buffer by expanding the bounding box
        minx, miny, maxx, maxy = shape_geom.bounds
        buffered_geom = box(
            minx - buffer_distance, 
            miny - (buffer_distance/2), 
            maxx + buffer_distance, 
            maxy + (buffer_distance/2)
        )
        
        # Get the corresponding TIFF file path
        tiff_filepath = os.path.join(input_dir, tiff_file)
        
        # Check if TIFF file exists
        if not os.path.exists(tiff_filepath):
            print(f"TIFF file {tiff_filepath} does not exist. Skipping.")
            continue
        
        # Open the TIFF file
        with rasterio.open(tiff_filepath) as src:
            # Get TIFF metadata
            tiff_bounds = src.bounds
            tiff_bounds_geom = box(*tiff_bounds)
            
            # Check if the buffered geometry intersects with the TIFF bounds
            if not buffered_geom.intersects(tiff_bounds_geom):
                print(f"Buffered shape {buffered_geom} does not intersect with TIFF {tiff_file}. Skipping.")
                continue
            
            # Adjust the buffered geometry to intersect with TIFF bounds
            buffered_geom = buffered_geom.intersection(tiff_bounds_geom)
            
            # Perform clipping
            output_filename = f"{os.path.splitext(shapefile_name)[0]}.tif"
            output_filepath = os.path.join(output_path, output_filename)
            clip_tiff_to_polygon_with_buffer(tiff_filepath, buffered_geom, output_filepath)
            
            # Calculate the transformed corners
            transform = src.transform
            corners = np.array([
                [0, 0], 
                [src.width, 0], 
                [src.width, src.height], 
                [0, src.height]
            ])
            transformed_corners = [transform * (x, y) for x, y in corners]
            
            # Create a bounding box from the transformed corners
            minx, miny = np.min(transformed_corners, axis=0)
            maxx, maxy = np.max(transformed_corners, axis=0)
            clipped_bounds = box(minx, miny, maxx, maxy)

            # Store results
            results.append({
                'Filename': output_filename,
                'MinX': clipped_bounds.bounds[0],
                'MinY': clipped_bounds.bounds[1],
                'MaxX': clipped_bounds.bounds[2],
                'MaxY': clipped_bounds.bounds[3],
                'Geometry': dumps(buffered_geom)
            })
    
    # Create DataFrame from results
    results_df = pd.DataFrame(results)
    
    print(f"TIFF files clipped and saved. Processed {len(results_df)} files.")
    return results_df

def clip_tiff_to_polygon_with_buffer(tiff_file, polygon_geom, output_file):
    """Clip a TIFF file to the specified polygon geometry with a buffer and save the result."""
    
    with rasterio.open(tiff_file) as src:
        # Mask the image to the polygon
        out_image, out_transform = rasterio.mask.mask(src, [polygon_geom], crop=True)
        
        # Update metadata
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        
        # Write the clipped raster to the output file
        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(out_image)
            
    print(f"Clipped TIFF saved to {output_file}")

def plot_shapefiles(folium_map, shapefile_dir):
    """Plot bounding polygons of shapefiles on the Folium map."""
    shapefile_list = glob.glob(os.path.join(shapefile_dir, '*.shp'))
    
    if not shapefile_list:
        raise ValueError("No shapefiles found in the directory.")
    
    latitudes = []
    longitudes = []
    
    # Plot shapefile polygons
    for shp_file in shapefile_list:
        gdf = gpd.read_file(shp_file)
        
        # Ensure CRS is in the right format for the transformer
        crs = gdf.crs.to_string()
        
        file_name = os.path.basename(shp_file)
        
        for _, row in gdf.iterrows():
            geom = shape(row['geometry'])
            
            if geom.geom_type == 'Polygon':
                # Extract boundary coordinates
                coords = list(geom.exterior.coords)
                
                # Convert to latitude and longitude
                lat_lon_coords = [utm_to_latlon(*coord, crs) for coord in coords]
                
                # Create a popup with the file name
                popup = folium.Popup(file_name, parse_html=True)
                
                # Plot the polygon on the Folium map
                folium.Polygon(
                    locations=lat_lon_coords,
                    color='black',
                    weight=2,
                    fill=True,
                    fill_color='blue',  # Color of the fill
                    popup=popup  # Add the popup to the polygon
                ).add_to(folium_map)
                
                # Add to list for center calculation
                latitudes.extend(lat for lat, _ in lat_lon_coords)
                longitudes.extend(lon for _, lon in lat_lon_coords)
    
    # Set map center to the mean latitude and longitude of all shapes
    if latitudes and longitudes:
        center_lat = sum(latitudes) / len(latitudes)
        center_lon = sum(longitudes) / len(longitudes)
        folium_map.location = [center_lat, center_lon]
        folium_map.zoom_start = 12  # Set the zoom level

def convert_tiffs_to_pngs_24bit(tiff_dir):
    """Convert TIFF files to 8-bit PNG format and save them in a subfolder within the TIFF directory."""
    # Create a subfolder for PNG files
    png_dir = os.path.join(tiff_dir, 'converted_pngs')
    os.makedirs(png_dir, exist_ok=True)
    
    # Get all TIFF files in the directory
    tiff_files = glob.glob(os.path.join(tiff_dir, '*.tif'))

    for tiff_file in tiff_files:
        print(f"Converting file: {tiff_file}")  # Debug: print the current TIFF file being processed
        
        # Define the path for the PNG file
        png_file = os.path.join(png_dir, os.path.basename(tiff_file).replace('.tif', '.png'))
        
        # Convert TIFF to PNG
        with rasterio.open(tiff_file) as src:
            # Read RGB bands
            red = src.read(1)
            green = src.read(2)
            blue = src.read(3)

            # Stack bands into a single 3D array (height x width x bands)
            rgb_array = np.stack([red, green, blue], axis=-1)
            
            # Normalize the image array to 8-bit (0-255)
            rgb_array = np.clip(rgb_array / np.max(rgb_array) * 255, 0, 255).astype(np.uint8)
            
            # Convert the array to a PNG image
            img = Image.fromarray(rgb_array)
            img.save(png_file, format='PNG', compress_level=9)
        
        print(f"Saved PNG file: {png_file}")  # Debug: print the path of the saved PNG file
    
    print(f"TIFF files converted to PNG and saved in {png_dir}")

def convert_tiffs_to_pngs_8bit(tiff_dir):
    """Convert TIFF files to 8-bit PNG format with transparency for black pixels and save them in a subfolder within the TIFF directory."""
    # Create a subfolder for PNG files
    png_dir = os.path.join(tiff_dir, 'converted_pngs')
    os.makedirs(png_dir, exist_ok=True)
    
    # Get all TIFF files in the directory
    tiff_files = glob.glob(os.path.join(tiff_dir, '*.tif'))

    for tiff_file in tiff_files:
        # Define the path for the PNG file
        png_file = os.path.join(png_dir, os.path.basename(tiff_file).replace('.tif', '.png'))
        
        # Convert TIFF to PNG
        with rasterio.open(tiff_file) as src:
            # Read RGB bands
            red = src.read(1)
            green = src.read(2)
            blue = src.read(3)

            # Normalize the bands to 0-255 range
            red_norm = np.clip((red / np.max(red)) * 255, 0, 255).astype(np.uint8)
            green_norm = np.clip((green / np.max(green)) * 255, 0, 255).astype(np.uint8)
            blue_norm = np.clip((blue / np.max(blue)) * 255, 0, 255).astype(np.uint8)

            # Create an alpha channel
            alpha = np.ones_like(red, dtype=np.uint8) * 255  # Default to fully opaque

            # Set alpha to 0 (transparent) where the pixel is black
            black_pixels = (red_norm == 0) & (green_norm == 0) & (blue_norm == 0)
            alpha[black_pixels] = 0

            # Stack bands into a single 4D array (height x width x bands)
            rgba_array = np.stack([red_norm, green_norm, blue_norm, alpha], axis=-1)
            
            # Convert the array to a PNG image
            img = Image.fromarray(rgba_array, 'RGBA')
            img.save(png_file, format='PNG', optimize=True, compress_level=9)
        
        print(f"Converted {tiff_file} to {png_file}")

def plot_tiffs(folium_map, clipped_tiff_df, tiff_dir, crs, type):
    """Plot PNG files as image overlays on the Folium map using bounds from the DataFrame and CRS."""

    for _, row in clipped_tiff_df.iterrows():
        tiff_file = row['Filename']
        # Change .tif to .png for PNG file or JPEG file
        if type == 'png':
            img_dir = tiff_dir + 'converted_pngs'  # Ensure proper directory separator
            img_file = img_dir + '/' + tiff_file.replace('.tif', '.png')
        elif type == 'jpeg':
            img_dir = tiff_dir + 'converted_jpegs'  # Ensure proper directory separator
            img_file = img_dir + '/' + tiff_file.replace('.tif', '.jpg')
        
        # Check if the PNG file exists
        if not os.path.exists(img_file):
            print(f"Image file {img_file} does not exist. Skipping.")
            continue

        # Parse the geometry from the WKT string
        geometry_wkt = row['Geometry']
        geom = wkt.loads(geometry_wkt)
        
        # Extract the bounding box coordinates
        minx, miny, maxx, maxy = geom.bounds
        
        # Convert UTM bounds to latitude and longitude
        lat_min, lon_min = utm_to_latlon(minx, miny, crs)
        lat_max, lon_max = utm_to_latlon(maxx, maxy, crs)
        
        # Define the bounds in latitude and longitude for the PNG overlay
        bounds = [
            [lat_min, lon_min],
            [lat_max, lon_max]
        ]
        
        # Add image as overlay
        print(f"Adding: {img_file} to map")
        folium.raster_layers.ImageOverlay(
            image=img_file,
            bounds=bounds,
            opacity=1,
            name=tiff_file,
            interactive=True,
            cross_origin=False
        ).add_to(folium_map)

    print("Image files added to Folium map.")

def tiff_remove_overlaps(input_dir):
    # Create the output directory inside the input directory
    output_dir = os.path.join(input_dir, 'without_overlap')
    os.makedirs(output_dir, exist_ok=True)
    
    # Read all TIFF files and store their data, bounds, and transforms
    tiff_data = []
    for filename in os.listdir(input_dir):
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            filepath = os.path.join(input_dir, filename)
            with rasterio.open(filepath) as src:
                data = src.read()  # Read all bands
                bounds = src.bounds
                tiff_data.append({
                    'filename': filename,
                    'data': data,
                    'bounds': bounds,
                    'transform': src.transform,
                    'meta': src.meta
                })
    
    # Initialize a list to store new bounding boxes and WKT strings
    new_bounds = []
    
    # Process each TIFF file
    for i, tiff in enumerate(tiff_data):
        print(f"Processing file: {tiff['filename']}")
        # Initialize mask as all True (i.e., keep all data initially)
        mask = np.ones_like(tiff['data'][0], dtype=bool)  # Single band mask
        
        for j, other_tiff in enumerate(tiff_data):
            if i != j:
                # Calculate the intersection (overlap) between the two bounding boxes
                intersection_bounds = rasterio.coords.BoundingBox(
                    left=max(tiff['bounds'].left, other_tiff['bounds'].left),
                    right=min(tiff['bounds'].right, other_tiff['bounds'].right),
                    bottom=max(tiff['bounds'].bottom, other_tiff['bounds'].bottom),
                    top=min(tiff['bounds'].top, other_tiff['bounds'].top)
                )
                
                if intersection_bounds.left < intersection_bounds.right and intersection_bounds.bottom < intersection_bounds.top:
                    # Get the window corresponding to the intersection
                    tiff_window = from_bounds(*intersection_bounds, tiff['transform'])
                    other_tiff_window = from_bounds(*intersection_bounds, other_tiff['transform'])
                    
                    # Convert window properties to integers for slicing
                    tiff_row_start, tiff_row_end = int(tiff_window.row_off), int(tiff_window.row_off + tiff_window.height)
                    tiff_col_start, tiff_col_end = int(tiff_window.col_off), int(tiff_window.col_off + tiff_window.width)
                    other_row_start, other_row_end = int(other_tiff_window.row_off), int(other_tiff_window.row_off + other_tiff_window.height)
                    other_col_start, other_col_end = int(other_tiff_window.col_off), int(other_tiff_window.col_off + other_tiff_window.width)
                    
                    # Handle cases where windows might be out of bounds
                    tiff_row_end = min(tiff_row_end, tiff['data'][0].shape[0])
                    tiff_col_end = min(tiff_col_end, tiff['data'][0].shape[1])
                    other_row_end = min(other_row_end, other_tiff['data'][0].shape[0])
                    other_col_end = min(other_col_end, other_tiff['data'][0].shape[1])
                    
                    # Read the overlapping regions for all bands
                    tiff_overlap = tiff['data'][:, tiff_row_start:tiff_row_end, tiff_col_start:tiff_col_end]
                    other_tiff_overlap = other_tiff['data'][:, other_row_start:other_row_end, other_col_start:other_col_end]
                    
                    # Create a mask for the overlap region
                    overlap_mask = np.zeros_like(tiff_overlap[0], dtype=bool)
                    
                    # Make sure the overlap_mask size matches the overlap data size
                    min_row, min_col = min(tiff_overlap.shape[1], other_tiff_overlap.shape[1]), min(tiff_overlap.shape[2], other_tiff_overlap.shape[2])
                    overlap_mask[:min_row, :min_col] = (tiff_overlap[0, :min_row, :min_col] > 0) & (other_tiff_overlap[0, :min_row, :min_col] > 0)
                    
                    # Update the full-sized mask for each band
                    mask[tiff_row_start:tiff_row_end, tiff_col_start:tiff_col_end] &= ~overlap_mask

        # Calculate new bounds after masking
        non_zero_indices = np.nonzero(mask)
        if non_zero_indices[0].size > 0 and non_zero_indices[1].size > 0:
            min_y, max_y = non_zero_indices[0].min(), non_zero_indices[0].max()
            min_x, max_x = non_zero_indices[1].min(), non_zero_indices[1].max()
            
            min_x_world = tiff['transform'][2] + min_x * tiff['transform'][0]
            min_y_world = tiff['transform'][5] + max_y * tiff['transform'][4]
            max_x_world = tiff['transform'][2] + max_x * tiff['transform'][0]
            max_y_world = tiff['transform'][5] + min_y * tiff['transform'][4]
            
            # Add to new_bounds list
            new_bounds_entry = {
                'Filename': tiff['filename'],
                'MinX': min_x_world,
                'MinY': min_y_world,
                'MaxX': max_x_world,
                'MaxY': max_y_world,
                'Geometry': box(min_x_world, min_y_world, max_x_world, max_y_world).wkt
            }
            new_bounds.append(new_bounds_entry)
            
            # Output the fields being saved to the DataFrame
            print(f"Saving to DataFrame: {new_bounds_entry}")
        
        # Save the processed TIFF file in the output directory
        output_filepath = os.path.join(output_dir, tiff['filename'])
        with rasterio.open(output_filepath, 'w', **tiff['meta']) as dst:
            for band in range(tiff['data'].shape[0]):
                band_data = tiff['data'][band] * mask
                dst.write(band_data.astype(np.uint8), band + 1)
    
    # Convert the list of new bounding boxes to a DataFrame
    new_bounds_df = pd.DataFrame(new_bounds)
    
    # Return the DataFrame with new bounding boxes
    return new_bounds_df

def convert_pngs_to_jpegs(input_dir, output_dir):
    """Convert PNG files in the input directory to JPEG format and save them in the output directory."""
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Get all PNG files in the input directory
    png_files = glob.glob(os.path.join(input_dir, '*.png'))
    
    for png_file in png_files:
        # Open the PNG image
        with Image.open(png_file) as img:
            # Define the output path for the JPEG file
            jpeg_file = os.path.join(output_dir, os.path.basename(png_file).replace('.png', '.jpg'))
            
            # Convert the image to RGB (required for JPEG)
            rgb_img = img.convert('RGB')
            
            # Save the image as JPEG
            rgb_img.save(jpeg_file, 'JPEG', quality=50)
        
        print(f"Converted {png_file} to {jpeg_file}")

In [2]:
# Example usage
BONA_ortho_directory = "C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/NEON_images-camera-ortho-mosaic/NEON.D19.BONA.DP3.30010.001.2019-08.basic.20240807T232622Z.RELEASE-2024/"
BONA_shapefile_dir_2 = 'C:/Users/allen/OneDrive/Desktop/Work/Scripts/Tree Segmentation/output/BONA_6km_segment_shapefiles'
BONA_clipped_tif_dir = "C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif/"
BONA_clipped_large_tif_dir = "C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/"

BONA_raw_file = "C:/Users/allen/OneDrive/Desktop/Work/Scripts/Tree Segmentation/output/20m_segment_coordinates.csv"
BONA_shapefile_csv = "C:/Users/allen/OneDrive/Desktop/Work/Scripts/small footprint/BONA_shapefiles.csv"
BONA_ortho_csv = "C:/Users/allen/OneDrive/Desktop/Work/Scripts/small footprint/BONA_orthophotos.csv"
BONA_shp_tif_search_csv = "C:/Users/allen/OneDrive/Desktop/Work/Scripts/small footprint/BONA_shp_tif_search.csv"
BONA_dead_pixel_count_csv = "C:/Users/allen/OneDrive/Desktop/Work/Scripts/small footprint/BONA_dead_pixel_count.csv"

# Define the CRS EPSG code for NEON BONA site in Alaska
BONA_CRS = 32606

In [3]:
BONA_shapefile_bounds_df = get_shapefile_bounds(BONA_shapefile_dir_2, BONA_shapefile_csv)
BONA_ortho_bounds_df = get_tiff_bounds(BONA_ortho_directory, BONA_ortho_csv)

BONA_shp_tif_search_results_df = check_shapefile_within_tiff(BONA_shapefile_bounds_df, BONA_ortho_bounds_df, BONA_shp_tif_search_csv)

Results written to C:/Users/allen/OneDrive/Desktop/Work/Scripts/small footprint/BONA_shapefiles.csv


C:\Users\allen\AppData\Local\Temp\ipykernel_20892\3075059551.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_temp], ignore_index=True)


Results written to C:/Users/allen/OneDrive/Desktop/Work/Scripts/small footprint/BONA_orthophotos.csv
Results written to C:/Users/allen/OneDrive/Desktop/Work/Scripts/small footprint/BONA_shp_tif_search.csv


In [4]:
print(BONA_shp_tif_search_results_df)

               Shapefile  Fully_Within_TIFF  \
0   segment_362131.0.shp               True   
1   segment_362135.0.shp               True   
2   segment_362138.0.shp               True   
3   segment_362187.0.shp               True   
4   segment_362203.0.shp               True   
..                   ...                ...   
72  segment_639639.0.shp               True   
73  segment_639641.0.shp               True   
74  segment_639646.0.shp               True   
75  segment_639667.0.shp               True   
76  segment_639670.0.shp               True   

                               TIFF_File  
0   2019_BONA_3_471000_7220000_image.tif  
1         2019_BONA_3_all_1m_UTM_geo.tif  
2   2019_BONA_3_471000_7221000_image.tif  
3   2019_BONA_3_471000_7222000_image.tif  
4   2019_BONA_3_471000_7222000_image.tif  
..                                   ...  
72  2019_BONA_3_474000_7221000_image.tif  
73  2019_BONA_3_474000_7221000_image.tif  
74  2019_BONA_3_474000_7221000_image.tif  
75  2

In [4]:
clipped_tif_df = clip_and_save_tiff(BONA_shapefile_bounds_df, BONA_ortho_directory, BONA_shp_tif_search_results_df, BONA_clipped_tif_dir)

Processing shapefile segment_362131.0.shp...
Clipping TIFF file: C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/NEON_images-camera-ortho-mosaic/NEON.D19.BONA.DP3.30010.001.2019-08.basic.20240807T232622Z.RELEASE-2024/2019_BONA_3_471000_7220000_image.tif
Output clipped TIFF path: C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif/clipped_segment_362131.0.shp.tif
Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif/clipped_segment_362131.0.shp.tif
Processing shapefile segment_362135.0.shp...
Clipping TIFF file: C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/NEON_images-camera-ortho-mosaic/NEON.D19.BONA.DP3.30010.001.2019-08.basic.20240807T232622Z.RELEASE-2024/2019_BONA_3_all_1m_UTM_geo.tif
Output clipped TIFF path: C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif/clipped_segment_362135.0.shp.tif
Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_t

In [6]:
print(len(BONA_shp_tif_search_results_df))

77


In [43]:
clipped_large_tif_df = clip_and_save_large_tiff(BONA_ortho_directory, BONA_shapefile_dir_2, BONA_shp_tif_search_results_df, BONA_clipped_large_tif_dir)

Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/segment_362131.0.tif
Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/segment_362135.0.tif
Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/segment_362138.0.tif
Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/segment_362187.0.tif
Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/segment_362203.0.tif
Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/segment_362204.0.tif
Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/segment_362209.0.tif
Clipped TIFF saved to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/segment_362270.0.tif
Clipped TIFF saved to C:/Users/allen/One

In [5]:
clipped_large_tif_without_overlap_df = tiff_remove_overlaps(BONA_clipped_large_tif_dir)

Processing file: segment_362131.0.tif
Saving to DataFrame: {'Filename': 'segment_362131.0.tif', 'MinX': 471400.5, 'MinY': 7220844.0, 'MaxX': 471713.8, 'MaxY': 7221000.0, 'Geometry': 'POLYGON ((471713.8 7220844, 471713.8 7221000, 471400.5 7221000, 471400.5 7220844, 471713.8 7220844))'}
Processing file: segment_362135.0.tif
Saving to DataFrame: {'Filename': 'segment_362135.0.tif', 'MinX': 471392.0, 'MinY': 7220924.0, 'MaxX': 471706.0, 'MaxY': 7221114.0, 'Geometry': 'POLYGON ((471706 7220924, 471706 7221114, 471392 7221114, 471392 7220924, 471706 7220924))'}
Processing file: segment_362138.0.tif
Saving to DataFrame: {'Filename': 'segment_362138.0.tif', 'MinX': 471386.6, 'MinY': 7221000.1, 'MaxX': 471700.5, 'MaxY': 7221173.5, 'Geometry': 'POLYGON ((471700.5 7221000.1, 471700.5 7221173.5, 471386.6 7221173.5, 471386.6 7221000.1, 471700.5 7221000.1))'}
Processing file: segment_362187.0.tif
Saving to DataFrame: {'Filename': 'segment_362187.0.tif', 'MinX': 471293.5, 'MinY': 7222000.100000001, '

In [58]:
# print(clipped_tif_df)
print(clipped_large_tif_without_overlap_df)

                Filename      MinX       MinY      MaxX       MaxY  \
0   segment_362131.0.tif  471400.5  7220844.0  471713.8  7221000.0   
1   segment_362135.0.tif  471392.0  7220924.0  471706.0  7221114.0   
2   segment_362138.0.tif  471386.6  7221000.1  471700.5  7221173.5   
3   segment_362187.0.tif  471293.5  7222000.1  471605.4  7222148.4   
4   segment_362203.0.tif  471264.1  7222296.4  471574.2  7222448.8   
..                   ...       ...        ...       ...        ...   
72  segment_639639.0.tif  474494.9  7221312.0  474807.3  7221484.2   
73  segment_639641.0.tif  474490.2  7221272.7  474802.5  7221444.2   
74  segment_639646.0.tif  474478.4  7221173.0  474790.8  7221344.7   
75  segment_639667.0.tif  474428.0  7220755.2  474740.4  7220926.9   
76  segment_639670.0.tif  474416.2  7220696.4  474728.6  7220867.5   

                                             Geometry  
0   POLYGON ((471713.8 7220844, 471713.8 7221000, ...  
1   POLYGON ((471706 7220924, 471706 7221114, 4

In [8]:
convert_tiffs_to_pngs_8bit(BONA_clipped_tif_dir)

Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif\clipped_segment_362131.0.shp.tif to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif/converted_pngs\clipped_segment_362131.0.shp.png
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif\clipped_segment_362135.0.shp.tif to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif/converted_pngs\clipped_segment_362135.0.shp.png
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif\clipped_segment_362138.0.shp.tif to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif/converted_pngs\clipped_segment_362138.0.shp.png
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif\clipped_segment_362187.0.shp.tif to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_tif/converted_pngs\clipped_segment_362187.0.shp.png
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orth

In [63]:
convert_tiffs_to_pngs_8bit(BONA_clipped_large_tif_dir)

Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif\segment_362131.0.tif to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/converted_pngs\segment_362131.0.png
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif\segment_362135.0.tif to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/converted_pngs\segment_362135.0.png
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif\segment_362138.0.tif to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/converted_pngs\segment_362138.0.png
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif\segment_362187.0.tif to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif/converted_pngs\segment_362187.0.png
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif\segment_362203.0.

In [64]:
convert_pngs_to_jpegs(BONA_clipped_large_tif_dir + "/converted_pngs/", BONA_clipped_large_tif_dir + "/converted_jpegs/")

Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif//converted_pngs\segment_362131.0.png to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif//converted_jpegs/segment_362131.0.jpg
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif//converted_pngs\segment_362135.0.png to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif//converted_jpegs/segment_362135.0.jpg
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif//converted_pngs\segment_362138.0.png to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif//converted_jpegs/segment_362138.0.jpg
Converted C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif//converted_pngs\segment_362187.0.png to C:/Users/allen/OneDrive/Desktop/Work/data/orthophotos/BONA/clipped_large_tif//converted_jpegs/segment_362187.0.jpg
Converted C:/Users/allen/OneDriv

In [50]:
print(clipped_large_tif_without_overlap_df)

                Filename      MinX       MinY      MaxX       MaxY  \
0   segment_362131.0.tif  471400.5  7220844.0  471713.8  7221000.0   
1   segment_362138.0.tif  471386.7  7221000.1  471700.5  7221173.5   
2   segment_362187.0.tif  471293.5  7222000.1  471605.4  7222148.3   
3   segment_362203.0.tif  471264.2  7222296.5  471574.2  7222448.8   
4   segment_362209.0.tif  480229.3  7222000.1  480541.1  7222097.2   
5   segment_362270.0.tif  474129.5  7224000.1  474441.4  7224108.3   
6   segment_362345.0.tif  476717.2  7224291.0  476999.9  7224462.5   
7   segment_639193.0.tif  475553.1  7230172.3  475865.4  7230344.5   
8   segment_639217.0.tif  475495.7  7229695.2  475808.0  7229866.8   
9   segment_639226.0.tif  475474.6  7229516.7  475786.8  7229688.3   
10  segment_639273.0.tif  478582.8  7228198.6  478895.2  7228370.8   
11  segment_639278.0.tif  472135.3  7228866.9  472447.6  7229000.0   
12  segment_639282.0.tif  478561.3  7228019.8  478873.6  7228192.1   
13  segment_639289.0

In [ ]:
# Initialize the Folium map object
folium_map = folium.Map(location=[0, 0], zoom_start=12, max_zoom=20)  # Placeholder initial location

# plot_tiffs(folium_map, clipped_tif_df, BONA_clipped_tif_dir, BONA_CRS)
plot_tiffs(folium_map, clipped_large_tif_without_overlap_df, BONA_clipped_large_tif_dir, BONA_CRS, "jpeg")

BONA_shpfile_dir_date = 'C:/Users/allen/OneDrive/Desktop/Work/Scripts/Tree Segmentation/output/BONA_6km_segment_shapefiles/2024_09_13'
# Plot shapefiles
plot_shapefiles(folium_map, BONA_shpfile_dir_date)

folium_map.save("BONA_6km_20m_segments_with_ortho_large.html")

folium_map

In [8]:
# Get the current working directory
current_directory = os.getcwd()

print("Current Working Directory:", current_directory)

Current Working Directory: c:\Users\allen\OneDrive\Desktop\Work\Scripts\small footprint


In [ ]:
# Initialize the Folium map object
folium_map = folium.Map(location=[0, 0], zoom_start=12, max_zoom=20)  # Placeholder initial location

BONA_raw_df = pd.read_csv(BONA_raw_file)

# Plot shapefiles
plot_shapefiles(folium_map, BONA_shapefile_dir_2)
plot_tiffs(folium_map, clipped_tif_df, BONA_clipped_tif_dir, BONA_CRS, "png")

folium_map.save("BONA_6km_20m_segments_with_ortho.html")

folium_map

In [78]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-allenqluo')

# Convert UTM to Lat/Lon
def utm_to_latlon(x, y, transformer):
    try:
        return transformer.transform(x, y)
    except Exception as e:
        print(f"Error transforming coordinates: {e}")
        raise

# Define the add_ee_layer function
def add_ee_layer(self, ee_image, vis_params, name):
    """Add Earth Engine layer to folium map."""
    map_id_dict = ee.Image(ee_image).getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

In [ ]:
wgs84_crs = "EPSG:4326"  # WGS84

# Create a transformer for converting UTM to Lat/Lon
try:
    transformer = Transformer.from_crs(BONA_CRS, wgs84_crs, always_xy=True)
except Exception as e:
    print(f"Error initializing transformer: {e}")
    raise

# Calculate global bounding box in UTM
min_x = BONA_shapefile_bounds_df['MinX'].min()
min_y = BONA_shapefile_bounds_df['MinY'].min()
max_x = BONA_shapefile_bounds_df['MaxX'].max()
max_y = BONA_shapefile_bounds_df['MaxY'].max()

# Convert UTM coordinates to latitude and longitude for the global bounding box
min_lon, min_lat = utm_to_latlon(min_x, min_y, transformer)
max_lon, max_lat = utm_to_latlon(max_x, max_y, transformer)

print(min_lat, min_lon, max_lat, max_lon)

# Define AOI as a GeoJSON polygon
aoi = ee.Geometry.Polygon([
    [
        [min_lon, min_lat],
        [max_lon, min_lat],
        [max_lon, max_lat],
        [min_lon, max_lat],
        [min_lon, min_lat]
    ]
])

print(aoi.getInfo())

# Load and filter the Landsat 8 image collection
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2019-06-01', '2019-07-01') \
    # .map(mask_clouds)

# Count the number of images in the collection
num_images = collection.size().getInfo()
print(f"Number of images in the collection: {num_images}")

collection = collection.map(apply_scale_factors)

# Take the median to reduce the effect of cloud cover
median_image = collection.median()

# Define visualization parameters
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # True color image (Red, Green, Blue)
    'min': 0.0,
    'max': 0.3,
}

# Add the Landsat 8 image layer to the map
add_ee_layer(map, median_image, vis_params, 'Landsat 8 Image')

# Add layer control to the map
folium.LayerControl().add_to(map)

# Display the map
map